In [1]:
import os

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent
import requests
from bs4 import BeautifulSoup

In [2]:
with open("api_key.txt", "r") as f:
    os.environ['OPENAI_API_KEY'] = f.read().strip()

In [3]:
prompt_template = """Write a LinkedIn post about {topic}, following these guidelines:
- Make the post concise and appealing
- Use emojis when suitable
"""

llm = OpenAI(temperature=0.7)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

print(llm_chain('Elon Musk')['text'])    


Elon Musk is one of the world's most inspiring entrepreneurs 🤩 His pioneering spirit and dedication to innovation have made him a role model for so many. From launching rockets 🚀 to creating revolutionary electric cars 🔋, his ambitions have changed the world. #ElonMusk #innovator #entrepreneur


In [4]:
def get_article(_):
    url = 'https://techcrunch.com/2023/10/02/elon-musk-is-being-sued-for-defamation-after-falsely-claiming-a-man-was-a-neo-nazi-on-x/'
    content = requests.get(url).text
    article = BeautifulSoup(content).find_all('div', {'class': 'article-content'})[0]
    return '\n\n'.join([p.text for p in article.find_all('p', recursive=False)])


elon_musk_retriever = Tool.from_function(
    func=get_article,
    name='Elon Musk Article Retriever',
    description='Useful when you want to retrieve an article about Elon Musk'
)

In [5]:
prompt_template = """Write a LinkedIn post based on the following article. Use these guidelines:
- Make the post concise and appealing
- Use emojis when suitable

{article}
"""

llm = OpenAI(temperature=0.7)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

linkedin_post_writer = Tool.from_function(
    func=lambda article: llm_chain(article)['text'],
    name='LinkedIn Post Writer',
    description='Useful when you want to write a LinkedIn post based on the given article.'
)

In [6]:
agent = initialize_agent(tools=[elon_musk_retriever, linkedin_post_writer],
                         llm=OpenAI(temperature=0),
                         verbose=True)

agent.run('Please write a LinkedIn post about Elon Musk')



> Entering new AgentExecutor chain...
 I need to find an article about Elon Musk
Action: Elon Musk Article Retriever
Action Input: Elon Musk
Observation: Elon Musk has been on something of a suing spree lately, but on Monday, Twitter’s new owner was hit with a fresh inbound lawsuit.

The lawsuit alleges that Musk defamed a California man after the owner of X (formerly Twitter) claimed that the recent college graduate was a federal agent operating undercover with a neo-Nazi organization in the Pacific Northwest.

Members of the group, known as the Rose City Nationalists, had their masks removed during a scuffle at an LGBTQ pride event in Oregon. When footage from the event appeared online, some X users incorrectly identified one of men involved in the incident as Benjamin Brody, a 22-year-old who belonged to a Jewish fraternity and previously remarked on the fraternity’s website that he wanted a job with the government.

Musk amplified the false claims on X at the time, describing the

'Elon Musk is being sued for defamation! 🤯 Unbelievable that he would make such a false claim against a recent college graduate who was an innocent bystander in the Pacific Northwest. 🤦\u200d♂️ #elonmusk #defamation #lawsuit'

In [7]:
current_date_url = "https://techcrunch.com/2023/10/02/"

def get_urls(topic):
    content = requests.get(current_date_url).text
    return "\n\n".join([a['href'] for a in BeautifulSoup(content).find_all(
        'a',
        {'class': 'post-block__title__link'}
    ) if topic in a.text])

topic_to_urls_tool = Tool.from_function(
    func=get_urls,
    name="Topic to URLs",
    description="Useful when urls are needed for a specific topic. You will be provided with a topic, and should return relevant urls"
)

In [8]:
def get_article(url):
    content = requests.get(url).text
    article = BeautifulSoup(content).find_all('div', {'class': 'article-content'})[0]
    return '\n\n'.join([p.text for p in article.find_all('p', recursive=False)])


url_to_article_tool = Tool.from_function(
    func=get_article,
    name='URL to Article',
    description='Useful when you want to retrieve an article from a url. You will be given a url and should return the article'
)

In [9]:
agent = initialize_agent(tools=[topic_to_urls_tool, url_to_article_tool, linkedin_post_writer],
                         llm=OpenAI(temperature=0),
                         verbose=True)

In [10]:
agent.run("Please write a LinkedIn post about Tesla")



> Entering new AgentExecutor chain...
 I need to find an article about Tesla
Action: Topic to URLs
Action Input: Tesla
Observation: https://techcrunch.com/2023/10/02/tesla-autopilot-arbitration-win-could-set-legal-benchmark-in-auto-industry/
Thought: I need to retrieve the article from the URL
Action: URL to Article
Action Input: https://techcrunch.com/2023/10/02/tesla-autopilot-arbitration-win-could-set-legal-benchmark-in-auto-industry/
Observation: In a victory for Tesla, a California federal judge ruled over the weekend that a group of Tesla owners cannot pursue in court claims that the company falsely advertised its automated features. Instead, they will have to face individual arbitration.

U.S. District Judge Haywood Gilliam’s ruling isn’t a win for the defensibility of Tesla’s advanced driver assistance systems, Autopilot and Full Self-Driving (FSD), but simply for Tesla’s terms and conditions. The plaintiffs who filed the proposed class action in September 2022 did in fact ag

'Tesla is revolutionizing the electric vehicle industry with its innovative technology and cutting-edge design. 🚗 The company is setting a new standard for the future of transportation! 🔋 #EVs #Innovation #FutureOfTransportation'